# Variational Auto-Encoders

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

from torch.utils.data import DataLoader

### Loading the MNIST Dataset

In [26]:
BATCH_SIZE = 100

# MNIST Dataset
train_dataset = datasets.MNIST(root="./mnist_data/", train = True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="./mnist_data/", train = False, transform=transforms.ToTensor(), download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle = False)


### VAE Class

In [27]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, x_dim, h_dim_1, h_dim_2, z_dim):
        super(VariationalAutoEncoder, self).__init__()

        self.x_dim = x_dim
        self.h_dim_1 = h_dim_1
        self.h_dim_2 = h_dim_2
        self.z_dim = z_dim

        self.fc1 = nn.Linear(x_dim, h_dim_1)
        self.fc2 = nn.Linear(h_dim_1, h_dim_2)
        self.fc31 = nn.Linear(h_dim_2, z_dim)
        self.fc32 = nn.Linear(h_dim_2, z_dim)


        self.fc4 = nn.Linear(z_dim, h_dim_2)
        self.fc5 = nn.Linear(h_dim_2, h_dim_1)
        self.fc6 = nn.Linear(h_dim_1, x_dim)

    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))

        return self.fc31(h), self.fc32(h)

    '''
    1. What is log_var.
    2. What is mu.

    Are they part of that parameterisation trick they were mentioning about?

    I think I get this now, somewhat.
    '''
    def sampling(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)

        return eps.mul(std).add(mu)

    def decoder(self, z):
        h = F.relu(self.fc4(z)) # Why not 2x here
        h = F.relu(self.fc5(h))

        return F.sigmoid(self.fc6(h))


    '''
    When we would do a backward pass, why wouldn't z be learnt - like mu and log_var
    '''
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))

        z = self.sampling(mu, log_var)

        return self.decoder(z), mu, log_var

### Model, Optimiser and Loss Function

In [28]:
vae = VariationalAutoEncoder(x_dim = 28 * 28, h_dim_1 = 512, h_dim_2 = 256, z_dim = 2)

# Is there any other way of setting the device?
if torch.cuda.is_available():
    vae.cuda()

optimiser = optim.Adam(vae.parameters())

def loss_function(recon_x, x, mu, log_var):
    x = x.view(x.size(0), -1)  # Flatten target to match recon_x shape

    BCE = F.binary_cross_entropy(recon_x, x, reduction="sum")

    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    return BCE + KLD

### Train

In [29]:
def train(epoch):
    vae.train()

    train_loss = 0

    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()

        optimiser.zero_grad()

        recon_batch, mu, log_var = vae(data)

        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()

        train_loss += loss.item() # .item() returns the value of the tensor as a standard python number

        optimiser.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

### Test

In [30]:
def test():
    vae.eval()
    test_loss = 0

    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()

            recon, mu, log_var = vae(data)

            # summing up the batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset) # Average
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [31]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.966172
Train Epoch: 1 [10000/60000 (17%)]	Loss: 188.750176
Train Epoch: 1 [20000/60000 (33%)]	Loss: 173.291309
Train Epoch: 1 [30000/60000 (50%)]	Loss: 168.363770
Train Epoch: 1 [40000/60000 (67%)]	Loss: 168.133945
Train Epoch: 1 [50000/60000 (83%)]	Loss: 159.189766
====> Epoch: 1 Average loss: 178.4591
====> Test set loss: 162.8736
Train Epoch: 2 [0/60000 (0%)]	Loss: 163.304434
Train Epoch: 2 [10000/60000 (17%)]	Loss: 162.468467
Train Epoch: 2 [20000/60000 (33%)]	Loss: 157.115332
Train Epoch: 2 [30000/60000 (50%)]	Loss: 164.491582
Train Epoch: 2 [40000/60000 (67%)]	Loss: 157.383037
Train Epoch: 2 [50000/60000 (83%)]	Loss: 161.157451
====> Epoch: 2 Average loss: 158.0970
====> Test set loss: 155.3253
Train Epoch: 3 [0/60000 (0%)]	Loss: 161.630283
Train Epoch: 3 [10000/60000 (17%)]	Loss: 144.600869
Train Epoch: 3 [20000/60000 (33%)]	Loss: 151.611514
Train Epoch: 3 [30000/60000 (50%)]	Loss: 156.518203
Train Epoch: 3 [40000/60000 (67%)]	Loss: 153.38

In [33]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './sample_' + '.png')